<a href="https://colab.research.google.com/github/mirwisek/AgrifarmPresentations/blob/master/Image_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
import time
start=time.time()
from keras.preprocessing.image import img_to_array
import pickle
import numpy as np
import cv2
from keras.layers import Dense, Activation, Conv2D, MaxPool2D, GlobalAvgPool2D, BatchNormalization, add, Input 
from keras.models import Model

In [0]:


# Complete Model
def ResNet50(input_shape, num_classes=10):
    input_object = Input(shape=input_shape)
    layers = [3, 4, 6, 3]
    channel_depths = [256, 512, 1024, 2048]

    output = Conv2D(64, kernel_size=7, strides=2, padding="same", kernel_initializer="he_normal")(input_object)
    output = BatchNormalization()(output)
    output = Activation("relu")(output)

    output = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(output)
    output = resnet_first_block_first_module(output, channel_depths[0])

    for i in range(4):
        channel_depth = channel_depths[i]
        num_layers = layers[i]

        strided_pool_first = True
        if (i == 0):
            strided_pool_first = False
            num_layers = num_layers - 1
        output = resnet_block(output, channel_depth=channel_depth, num_layers=num_layers,
                              strided_pool_first=strided_pool_first)

    output = GlobalAvgPool2D()(output)
    output = Dense(num_classes)(output)
    output = Activation("softmax")(output)

    model = Model(inputs=input_object, outputs=output)

    return model

In [19]:

def resnet_first_block_first_module(input, channel_depth):
    residual_input = input
    stride = 1

    residual_input = Conv2D(channel_depth, kernel_size=1, strides=1, padding="same", kernel_initializer="he_normal")(
        residual_input)
    residual_input = BatchNormalization()(residual_input)

    input = Conv2D(int(channel_depth / 4), kernel_size=1, strides=stride, padding="same",
                   kernel_initializer="he_normal")(input)
    input = BatchNormalization()(input)
    input = Activation("relu")(input)

    input = Conv2D(int(channel_depth / 4), kernel_size=3, strides=stride, padding="same",
                   kernel_initializer="he_normal")(input)
    input = BatchNormalization()(input)
    input = Activation("relu")(input)

    input = Conv2D(channel_depth, kernel_size=1, strides=stride, padding="same", kernel_initializer="he_normal")(input)
    input = BatchNormalization()(input)

    input = add([input, residual_input])
    input = Activation("relu")(input)

    return input
def resnet_block(input, channel_depth, num_layers, strided_pool_first=False):
    for i in range(num_layers):
        pool = False
        if (i == 0 and strided_pool_first):
            pool = True
        input = resnet_module(input, channel_depth, strided_pool=pool)

    return input
def resnet_module(input, channel_depth, strided_pool=False):
    residual_input = input
    stride = 1

    if (strided_pool):
        stride = 2
        residual_input = Conv2D(channel_depth, kernel_size=1, strides=stride, padding="same",
                                kernel_initializer="he_normal")(residual_input)
        residual_input = BatchNormalization()(residual_input)

    input = Conv2D(int(channel_depth / 4), kernel_size=1, strides=stride, padding="same",
                   kernel_initializer="he_normal")(input)
    input = BatchNormalization()(input)
    input = Activation("relu")(input)

    input = Conv2D(int(channel_depth / 4), kernel_size=3, strides=1, padding="same", kernel_initializer="he_normal")(
        input)
    input = BatchNormalization()(input)
    input = Activation("relu")(input)

    input = Conv2D(channel_depth, kernel_size=1, strides=1, padding="same", kernel_initializer="he_normal")(input)
    input = BatchNormalization()(input)

    input = add([input, residual_input])
    input = Activation("relu")(input)

    return input


# Prediction Function
def Make_prediction(image_path):
  try:
        image_list=[]

        # Read Image
        image = cv2.imread(image_path)
        if image is not None :

            # Resize Image
            image = cv2.resize(image, tuple((256,256)))

            image_list.append(img_to_array(image))
            
            image=np.array(image_list, dtype=np.float16) / 225.0 

            #Create Model
            model = ResNet50(input_shape=(256, 256, 3), num_classes=31)
            
            #Load Train Model Weights
            model.load_weights("/content/drive/My Drive/Plant_Disease_Model/idenprof_weight_model.009-0.554131031036377.h5")
            
            # Load Label_Binarizer 
            with open('/content/drive/My Drive/Plant_Disease_Model/label_transform.pkl', 'rb') as f:
              label = pickle.load(f)
              
              # Return Prediction
              return label.inverse_transform(model.predict(image))[0]
  except Exception as e:
      print(f"Error : {e}")
      return None

# Prediction Function Call
print(Make_prediction("/content/drive/My Drive/FYP/segmented/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335_final_masked.jpg"))
print("Time took to load: ", (time.time() - start))


None
Time took to load:  3.611846446990967


By Mirwise -- Creating TFLite file for MLKit

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Create Model
model = ResNet50(input_shape=(256, 256, 3), num_classes=31)

#Load Train Model Weights
model.load_weights("/content/drive/My Drive/Colab Notebooks/Resnet/idenprof_weight_model.009-0.554131031036377.h5")

In [21]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_55 (Conv2D)              (None, 128, 128, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_55 (BatchNo (None, 128, 128, 64) 256         conv2d_55[0][0]                  
__________________________________________________________________________________________________
activation_52 (Activation)      (None, 128, 128, 64) 0           batch_normalization_55[0][0]     
____________________________________________________________________________________________

In [0]:
import tensorflow as tf
run_model = tf.function(lambda x : model(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

In [30]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tfmodel = converter.convert() 
open("model_tf.tflite" , "wb").write(tfmodel)

ValueError: ignored